In [20]:
import os

import numpy as np
import sklearn

from utils.utils import read_dataset
from utils.utils import create_directory

In [21]:
def fit_classifier():
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
    #print('Number of Classes: %s' % nb_classes)
    
    #one-hot-encoding
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
    
    # save orignal y because later we will use binary
    y_true = np.argmax(y_test, axis=1) #See if this is really needed later
    
    if len(x_train.shape) == 2: #if univariate, check to see if this may make things harder later on
        #adds dimension making it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        
    input_shape = x_train.shape
    print(input_shape)
    classifier = create_classifier(classifier_name, input_shape, nb_classes, output_dir, True)
    
    classifier.fit(x_train, y_train, x_test, y_test, y_true)
    

In [22]:
def create_classifier(classifier_name, input_shape, nb_classes, output_dir, verbose=False):
    if classifier_name == 'mlp':
        from models import mlp
        return mlp.Classifier_MLP(output_dir, input_shape, nb_classes, verbose)
    if classifier_name == 'lstmfcn':
        from models import lstmfcn
        return lstmfcn.Classifier_LSTMFCN(output_dir, input_shape, nb_classes,  verbose)
    if classifier_name == 'emn':
        return Classifier_EMN(output_dir, nb_classes,input_shape, verbose)

In [23]:
root_dir_win = 'C:/Users/worf9_000/Desktop/bs-thesis/experiments'
root_dir_arch = '/home/worf/Work/bs-thesis/experiments'

root_dir = os.getcwd()

dataset_name = 'ShapesAll'
classifier_name = 'emn'

datasets_dict = read_dataset(root_dir, dataset_name)

In [26]:
output_dir = root_dir + '/results/' + classifier_name + '/UCRArchive_2018/' + dataset_name + '/'

test_dir_df_metrics = output_dir + 'df_metrics.csv'

print('Method: ', dataset_name, classifier_name)

if os.path.exists(test_dir_df_metrics):
    print('Already done')
#else:
    
create_directory(output_dir)
datasets_dict = read_dataset(root_dir, dataset_name)
    
fit_classifier()
    
print('DONE')
    
create_directory(output_dir + '/DONE')

Method:  ShapesAll emn
(600, 512, 1)
[0.6, 0.7]
512
[307, 358]
600


ValueError: Input 0 of layer conv2d_5 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [None, 32]

In [25]:
import tensorflow.keras as keras
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import time

from layers.reservoir import Reservoir

from utils.utils import save_logs

import matplotlib

from tqdm.keras import TqdmCallback

class Classifier_EMN:
    def __init__(self, output_dir, nb_classes, input_shape, verbose):
        self.output_dir = output_dir
        self.nb_classes = nb_classes
        self.verbose = verbose
        
        #Hyperparameters ESN
        self.esn_config = {'units':32, 'connect':0.7,'IS':0.1,"spectral":0.9,'leaky':1}
        
        #Hyperparameters Convolutions
        self.conv_config = {'epoch':500,'batch':25,'ratio':[0.6,0.7]}
        
        self.model = self.build_model(input_shape, nb_classes, input_shape[1])
        self.model.summary()
        
        
    def build_model(self, input_shape, nb_classes, len_series):
        ratio = self.conv_config['ratio']
        print(ratio)
        print(len_series)
        nb_rows = [np.int(ratio[0]*len_series),np.int(ratio[1]*len_series)]
        print(nb_rows)
        nb_cols = input_shape[0]
        print(nb_cols)
        
        input_layer = keras.layers.Input(input_shape[1:])
        
        res_layer = tfa.layers.ESN(units=32,connectivity=0.7,leaky=1,spectral_radius=0.9,use_bias=False)(input_layer)
        
        x_layer_1 = keras.layers.Conv2D(120, (nb_rows[0], nb_cols), kernel_initializer='lecun_uniform', activation='relu',
                                       padding='valid', strides=(1,1), data_format = 'channels_last')(res_layer)
        x_layer_1 = keras.layers.GlobalMaxPooling2D(data_format = 'channels_first')(x_layer_1)
        
        
        
        y_layer_1 = keras.layers.Conv2D(120, (nb_rows[1], nb_cols), kernel_initializer='lecun_uniform', activation='relu',
                                       padding='valid', strides=(1,1), data_format = 'channels_last')(res_layer)
        y_layer_1 = keras.layers.GlobalMaxPooling2D(data_format = 'channels_last')(y_layer_1)
        
        
        
        concat_layer = keras.layers.concatenate([x_layer_1, y_layer_1])
        concat_layer = keras.layers.Dropout(0.25)(concat_layer)
        
        output_layer = keras.layers.Dense(nb_classes, kernel_initializer='lecun_uniform', activation='softmax')(concat_layer)
        
        model = keras.models.Model(input_layer, output_layer)
        
        model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
        
        self.callbacks = [TqdmCallback(verbose=0)]
        
        
        return model
        
    def reshape_shuffle(self, x_train, y_train, nb_samples, res_units, len_series):

        #Generate template for train data
        train_data = np.zeros((nb_samples, 1, len_series, res_units))
        train_labels = np.zeros((nb_samples, self.nb_classes))

        #Generate Shuffle template
        L_train = [x_train for x_train in range(nb_samples)] #Array with size==samples, every value==index 
        np.random.shuffle(L_train)
        
        #For every series -> shuffle train and labels
        for m in range(nb_samples):
            train_data[m,0,:,:] = x_train[L_train[m],:,:] 
            train_labels[m,:] = y_train[L_train[m],:]
        
        return train_data, train_labels      
    
    def ff_esn(self, x_train, x_test):
        units = self.esn_config['units']
        connectivity = self.esn_config['connect']
        IS = self.esn_config['IS']
        spectral_radius = self.esn_config['spectral']
        leaky = self.esn_config['leaky']
        n_in = 1
        
        escnn = Reservoir(units, n_in, IS, spectral_radius, connectivity, leaky)
        echoes_train = escnn.set_weights(x_train)
        echoes_test = escnn.set_weights(x_test)
        
        return echoes_train, echoes_test
        
    def fit(self, x_train, y_train, x_val, y_val, y_true):        
        
        #3. Train Model
        batch = self.conv_config['batch']
        epoch = self.conv_config['epoch']
        
        start_time = time.time()
        
        hist = self.model.fit(x_train, y_train, batch_size=batch, epochs=epoch,
            verbose=False, validation_data=(x_val,y_val), callbacks=self.callbacks)
        
        duration = time.time() - start_time
        
        y_pred = self.model.predict(x_val)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred , axis=1)

        save_logs(self.output_dir, hist, y_pred, y_true, duration, self.verbose, lr=False)

        keras.backend.clear_session()